In [62]:
import pandas as pd

In [190]:
lang = 'en'
#multicardioner_track2_cardioccc_dev_es_predictions_joined
mentions_file = f'output/multicardioner_track2_test_{lang}_predictions_xlmr_medical_True_joined.tsv'
external_mentions_file = f'output/extracted_drugs_test_{lang}_filtered.tsv'
result_file = f'output/multicardioner_track2_test_{lang}_predictions_xlmr_medical_True_joined_combined.tsv'

In [191]:
df_mentions = pd.read_csv(mentions_file, sep='\t', header=0, index_col=False)
df_mentions.head()

,filename,label,start_span,end_span,text
0,multicardioner_test+bg_100,FARMACO,792,799,oxígeno
1,multicardioner_test+bg_100,FARMACO,850,857,oxígeno
2,multicardioner_test+bg_100,FARMACO,1191,1200,fentanilo
3,multicardioner_test+bg_100,FARMACO,1719,1728,midazolam
4,multicardioner_test+bg_100,FARMACO,1738,1745,morfina


In [203]:
df_external_mentions = pd.read_csv(external_mentions_file, sep='\t', header=0, index_col=False)
df_external_mentions.head()

,file_name,label,start_index,end_index,drug_in_text,reference,original_drug,context
0,multicardioner_test+bg_999,FARMACO,946,955,imiquimod,Arbol_Medicamentos_DSCA_Spanish-D06BB10-CIMAMe...,imiquimod,"CONTEXT:coids, retinoids, imiquimod, local rad..."
1,multicardioner_test+bg_999,FARMACO,977,993,interferon alpha,MultiCardioNER,interferon alpha,"CONTEXT:cal radiotherapy, interferon alpha or ..."
2,multicardioner_test+bg_999,FARMACO,1020,1032,methotrexate,DrugBank-DrugCentral-MedList-MultiCardioNER-NI...,methotrexate,"CONTEXT:, oral retinoids, methotrexate and oth..."
3,multicardioner_test+bg_998,FARMACO,3001,3011,epirubicin,DrugBank-DrugCentral-MultiCardioNER,epirubicin,CONTEXT:py treatment with epirubicin 90 mg/m2 ...
4,multicardioner_test+bg_998,FARMACO,3025,3041,cyclophosphamide,DrugBank-DrugCentral-MedList-MultiCardioNER-NI...,cyclophosphamide,CONTEXT:icin 90 mg/m2 and cyclophosphamide 600...


In [204]:
# filter drug bank
# remove start/end whitespaces and adjust indices
for index, row in df_external_mentions.iterrows():
    try:
        start = int(row['start_index'])
        end  = int(row['end_index'])
    except:
        print(index, row)

In [205]:
df_external_mentions = df_external_mentions.astype({'start_index': 'int64', 'end_index': 'int64'})

In [206]:
df_external_mentions['drugbank'] = df_external_mentions['reference'].apply(lambda x: 'DrugBank' in str(x))

In [207]:
df_external_mentions_filter = df_external_mentions[~df_external_mentions['drugbank']]
df_external_mentions_filter.head()

,file_name,label,start_index,end_index,drug_in_text,reference,original_drug,context,drugbank
1,multicardioner_test+bg_999,FARMACO,977,993,interferon alpha,MultiCardioNER,interferon alpha,"CONTEXT:cal radiotherapy, interferon alpha or ...",False
8,multicardioner_test+bg_996,FARMACO,94,101,alcohol,CIMAMedSpanish,alcohol,"CONTEXT:rn of dependence, alcohol abuse since ...",False
14,multicardioner_test+bg_991,FARMACO,610,625,Immunoglobulins,MultiCardioNER,immunoglobulins,"CONTEXT:e) were negative. Immunoglobulins IgA,...",False
17,multicardioner_test+bg_990,FARMACO,2363,2370,Alcohol,CIMAMedSpanish,alcohol,CONTEXT:5 cigarettes/day.Alcohol consumption: ...,False
18,multicardioner_test+bg_990,FARMACO,2393,2400,alcohol,CIMAMedSpanish,alcohol,CONTEXT:umption: Drinks 5 alcoholic drinks on ...,False


In [208]:
df_external_mentions_filter.shape

(7568, 9)

In [209]:
df_external_mentions.shape

(34523, 9)

In [210]:
for index, row in df_external_mentions.iterrows():
    mention_text = str(row['drug_in_text'])
    left_whitespace = mention_text.lstrip()
    if len(mention_text) - len(left_whitespace) > 0:
        df_external_mentions.at[index, 'start_index'] = row['start_index'] + (len(mention_text) - len(left_whitespace))
        
    right_whitespace = mention_text.rstrip()
    if len(mention_text) - len(right_whitespace) > 0:
        df_external_mentions.at[index, 'end_index'] = row['end_index'] - (len(mention_text) - len(right_whitespace))
        
    if (len(mention_text) - len(left_whitespace) > 0) or (len(mention_text) - len(right_whitespace) > 0):
        df_external_mentions.at[index, 'drug_in_text'] = mention_text.strip()

In [211]:
def filter_spans(spans):
    get_sort_key = lambda span: (span['end_span'] - span['start_span'], -span['start_span'])
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens: Set[int] = set()
    for span in sorted_spans:
        # Check for end - 1 here because boundaries are inclusive
        if span['start_span'] not in seen_tokens and span['end_span'] - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span['start_span'], span['end_span']))
    result = sorted(result, key=lambda span: span['start_span'])
    return result

In [212]:
external_filenames = list(df_external_mentions['file_name'].values)
original_filenames = list(df_mentions['filename'].values)
original_filenames.extend(external_filenames)
filenames = list(set(original_filenames))
filenames.sort()

In [216]:
all_spans = []
for filename in filenames:
    # get both mentions
    df_mentions_file = df_mentions[df_mentions['filename'] == filename]
    df_external_mentions_file = df_external_mentions[df_external_mentions['file_name'] == filename]
    # merge spans
    spans = []

    for index, mention in df_mentions_file.iterrows():
        spans.append({
            'filename': mention['filename'],
            'label': mention['label'],
            'start_span': mention['start_span'],
            'end_span': mention['end_span'],
            'text': mention['text']
        })

    for index, mention in df_external_mentions_file.iterrows():
        spans.append({
            'filename': mention['file_name'],
            'label': mention['label'],
            'start_span': mention['start_index'],
            'end_span': mention['end_index'],
            'text': mention['drug_in_text']
        })

    filtered_spans = filter_spans(spans)
    all_spans.extend(filtered_spans)

In [217]:
df_all_mentions = pd.DataFrame.from_records(all_spans)
df_all_mentions.head()

,filename,label,start_span,end_span,text
0,multicardioner_test+bg_10,FARMACO,1541,1547,Oxygen
1,multicardioner_test+bg_10,FARMACO,2678,2685,balance
2,multicardioner_test+bg_100,FARMACO,674,680,oxygen
3,multicardioner_test+bg_100,FARMACO,730,736,oxygen
4,multicardioner_test+bg_100,FARMACO,792,799,oxígeno


In [218]:
df_all_mentions[['filename', 'label', 'start_span', 'end_span', 'text']].to_csv(result_file, sep='\t', index=False)

In [ ]:
# en - |0.8475|0.9076|0.8765 - combined w/ drugbank, |0.8732|0.9|0.8864 - w/o drugbank
# es - 0.6252|0.9092|0.7409 - combined w/ drugbank, |0.6281|0.9096|0.743 - w/o drugbank
# it ??